# 08 Add LSOA Code and Postcode Centroids to NHS Facility CSVs

This notebook enriches NHS facility datasets by assigning:
- **LSOA 2021 codes (`lsoa21cd`)**
- **Postcode centroid coordinates (`lat`, `long`)**

Each facility is matched using its recorded postcode against the official ONS postcode lookup file.

## Tasks:
- Clean and standardise postcode fields
- Merge each facility dataset with postcode lookup
- Add `lsoa21cd`, `lat`, and `long` columns to each row
- Save enriched outputs to a new subdirectory

## Inputs:
- NHS facility CSV files:
  - Located in `data/processed/Health Infrastructure/`
  - Files include:
    - `NHS_SW_Community_Diagnostic_Centres.csv`
    - `NHS_SW_GP_Practices_and_Polyclinics.csv`
    - `NHS_SW_Acute_Hospitals.csv`
    - `NHS_SW_Ambulance_Stations.csv`
    - `NHS_SW_Community_and_Outreach_Facilities.csv`
    - `NHS_SW_Community_Hospitals.csv`
    - `NHS_SW_Specialist_Hospitals.csv`
- Postcode lookup file:
  - `data/raw/PCD_OA21_LSOA21_MSOA21_LAD_MAY24_UK_LU.csv`

## Outputs:
- Enriched CSVs with added `lsoa21cd`, `lat`, and `long` columns
- Files saved to: `data/processed/Health Infrastructure/enriched/`
- Optional: summary of matched and unmatched postcode counts per file


In [14]:
import pandas as pd
from pathlib import Path

# Set paths
input_dir = Path("../data/processed/Health Infrastructure")
lookup_path = Path("../data/raw/PCD_OA21_LSOA21_MSOA21_LAD_MAY24_UK_LU.csv")
output_dir = input_dir / "enriched"
output_dir.mkdir(exist_ok=True)
# Load postcode-to-geography lookup with correct encoding
lookup = pd.read_csv(lookup_path, dtype=str, encoding="latin1")
lookup["pcd_clean"] = lookup["pcds"].str.strip().str.upper().str.replace(" ", "")

# Select required columns
lookup = lookup[["pcd_clean", "lsoa21cd", "msoa21cd", "ladcd"]]

# Confirm columns loaded
print("Lookup file columns:", lookup.columns.tolist())

Lookup file columns: ['pcd_clean', 'lsoa21cd', 'msoa21cd', 'ladcd']


In [15]:
# Preview files before enrichment
summary_rows = []
for file in sorted(input_dir.glob("*.csv")):
    try:
        df = pd.read_csv(file, dtype=str, nrows=5)
        summary_rows.append({
            "Filename": file.name,
            "Preview Rows": len(df),
            "Columns": ", ".join(df.columns)
        })
    except Exception as e:
        summary_rows.append({
            "Filename": file.name,
            "Preview Rows": "ERROR",
            "Columns": f"Error reading file: {e}"
        })

summary_df = pd.DataFrame(summary_rows)
display(summary_df)

,Filename,Preview Rows,Columns
0,NHS_SW_ Community_Diagnostic_Centres.csv,5,"Code, Name, Address 1, Address 2, Address 3, A..."
1,NHS_SW_ GP_Practices_and_Polyclinics.csv,5,"Code, Name, Address 1, Address 2, Address 3, A..."
2,NHS_SW_Acute_Hospitals.csv,5,"Code, Name, Address 1, Address 2, Address 3, A..."
3,NHS_SW_Ambulance_Stations.csv,5,"Code, Name, Address 1, Address 2, Address 3, A..."
4,NHS_SW_Community_Hospitals.csv,5,"Code, Name, Address 1, Address 2, Address 3, A..."
5,NHS_SW_Community_and_Outreach_Facilities.csv,5,"Community, Name, Address 1, Address 2, Address..."
6,NHS_SW_Specialist_Hospitals.csv,5,"Code, Name, Address 1, Address 2, Address 3, A..."


In [16]:
# Enrichment process
for file in sorted(input_dir.glob("*.csv")):
    print(f"\nProcessing: {file.name}")

    try:
        df = pd.read_csv(file, dtype=str)

        if "Postcode" not in df.columns:
            print("Expected column 'Postcode' not found — skipping.")
            continue

        # Clean postcode
        df["POSTCODE_CLEAN"] = df["Postcode"].astype(str).str.strip().str.upper().str.replace(" ", "")

        # Merge with lookup on postcode
        merged = pd.merge(
            df,
            lookup,
            left_on="POSTCODE_CLEAN",
            right_on="pcd_clean",
            how="left"
        )

        # Match stats
        total = len(merged)
        matched = merged["lsoa21cd"].notna().sum()
        print(f"Matched: {matched} / {total} ({matched / total:.1%})")

        # Save enriched file
        output_file = output_dir / f"{file.stem}_enriched.csv"
        merged.to_csv(output_file, index=False)
        print(f"Saved: {output_file.name}")

    except Exception as e:
        print(f"Error processing {file.name}: {e}")


Processing: NHS_SW_ Community_Diagnostic_Centres.csv
Matched: 24 / 24 (100.0%)
Saved: NHS_SW_ Community_Diagnostic_Centres_enriched.csv

Processing: NHS_SW_ GP_Practices_and_Polyclinics.csv
Matched: 963 / 963 (100.0%)
Saved: NHS_SW_ GP_Practices_and_Polyclinics_enriched.csv

Processing: NHS_SW_Acute_Hospitals.csv
Matched: 24 / 264 (9.1%)
Saved: NHS_SW_Acute_Hospitals_enriched.csv

Processing: NHS_SW_Ambulance_Stations.csv
Matched: 87 / 87 (100.0%)
Saved: NHS_SW_Ambulance_Stations_enriched.csv

Processing: NHS_SW_Community_Hospitals.csv
Matched: 92 / 92 (100.0%)
Saved: NHS_SW_Community_Hospitals_enriched.csv

Processing: NHS_SW_Community_and_Outreach_Facilities.csv
Matched: 278 / 278 (100.0%)
Saved: NHS_SW_Community_and_Outreach_Facilities_enriched.csv

Processing: NHS_SW_Specialist_Hospitals.csv
Matched: 23 / 23 (100.0%)
Saved: NHS_SW_Specialist_Hospitals_enriched.csv
